# 第五章 深层学习模型

本章依赖的类库有：

- numpy 快速操作结构数组的工具
- keras 二次封装的深度学习库
- tensorflow 深度学习库
- caffe 深度学习库

python类库安装教程见实体书附录A，深度学习库安装教程见实体书附录B，每个小节可以独立运行。
____

## 5.1 解密生物智能

### 实验一：大脑的材料

In [1]:
import sys

# 保存当前重定向
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
# 修改默认的ascii编码
sys.setdefaultencoding('utf-8')
# 恢复重定向
sys.stdout = default_stdout
sys.stderr = default_stderr

"""由五感输入的刺激信号。对应行为：看 听 闻 触 嗅"""
d_signals = {
    '视觉信号': '看',
    '听觉信号': '听'
}


class Neuron(object):
    """神经元"""

    def __init__(self, signal_type):
        # 神经元拥有处理某种类型信号的能力
        self.signal_type = signal_type

    def spike(self, x):
        """神经元激活函数。输入某种类型的刺激信号，有可能激活神经元响应刺激"""
        if x.signal_type == self.signal_type:
            return d_signals[self.signal_type] + '：' + x.data
        else:
            return d_signals[self.signal_type] + '：' + '什么都没' + \
                   d_signals[self.signal_type] + '到'


class SignalInput(object):
    """输入信号"""
    
    def __init__(self, signal_type, data):
        self.signal_type = signal_type
        self.data = data


x = SignalInput('视觉信号', '一只猫在卖萌!')
print(Neuron('视觉信号').spike(x))

看：一只猫在卖萌!


### 实验二：探索脑皮层的功能区域

In [2]:
from collections import Counter


class VisualBrain:
    """视觉皮层"""

    def __init__(self, num):
        self.neurons = [Neuron('视觉信号') for i in range(num)]

    def process(self, x):
        """处理信号"""
        print('视觉皮层在处理：' + x.data)


class AuditoryBrain:
    """听觉皮层"""

    def __init__(self, num):
        self.neurons = [Neuron('听觉信号') for i in range(num)]

    def process(self, x):
        """处理信号"""
        print('听觉皮层在处理：' + x.data)


class Brain:
    """脑皮层"""

    def __init__(self):
        # 分配神经元数量，人脑更依赖视觉输入获得信息，所以视觉神经元数量应该更多
        self.visual_model = VisualBrain(3000)
        self.auditory_model = AuditoryBrain(1000)

    def process(self, x):
        """根据不同的传入信号，传递给不同的皮层组织处理"""
        result = {
            '视觉信号': lambda x: self.visual_model.process(x),
            '听觉信号': lambda x: self.auditory_model.process(x),
        }[x.signal_type](x)


brain = Brain()
x_see = SignalInput('视觉信号', '一只猫在卖萌!')
x_hear = SignalInput('听觉信号', '猫咪在喵喵叫!')

brain.process(x_see)
brain.process(x_hear)

视觉皮层在处理：一只猫在卖萌!
听觉皮层在处理：猫咪在喵喵叫!


### 实验三：不同的皮层组织——区别在于函数算法

In [3]:
class VisualBrain:
    """视觉皮层"""

    def __init__(self, num):
        self.neurons = [Neuron('视觉信号') for i in range(num)]

    def process(self, x):
        """处理信号"""
        print('输入' + x.signal_type + '：' + x.data)
        reactions = [neuron.spike(x) for neuron in self.neurons]
        # 一群神经元集体响应投票，打印票数最高的响应
        top_reaction = Counter(reactions).most_common(1)
        print('>> 我在用眼睛（视觉皮层）' + top_reaction[0][0])


class AuditoryBrain:
    """听觉皮层"""

    def __init__(self, num):
        self.neurons = [Neuron('听觉信号') for i in range(num)]

    def process(self, x):
        """处理信号"""
        print('输入' + x.signal_type + '：' + x.data)
        reactions = [neuron.spike(x) for neuron in self.neurons]
        # 一群神经元集体响应投票，打印票数最高的响应
        top_reaction = Counter(reactions).most_common(1)
        print('>> 我在用耳朵（听觉皮层）' + top_reaction[0][0])

In [4]:
brain = Brain()
x_see = SignalInput('视觉信号', '一只猫在卖萌!')
x_hear = SignalInput('听觉信号', '猫咪在喵喵叫!')

brain.process(x_see)
brain.process(x_hear)

输入视觉信号：一只猫在卖萌!
>> 我在用眼睛（视觉皮层）看：一只猫在卖萌!
输入听觉信号：猫咪在喵喵叫!
>> 我在用耳朵（听觉皮层）听：猫咪在喵喵叫!


In [5]:
brain.auditory_model.process(x_see)
brain.visual_model.process(x_hear)

输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入听觉信号：猫咪在喵喵叫!
>> 我在用眼睛（视觉皮层）看：什么都没看到


### 实验四：可替换的皮层模块——神经元组成的学习模型

In [6]:
import random


class Neuron(object):
    """神经元"""

    def __init__(self, signal_type):
        # 神经元拥有处理某种类型信号的能力
        self.signal_type = signal_type

    def spike(self, x):
        """输入某种类型的刺激信号，有可能激活神经元响应刺激"""
        # 假设神经元有1%的概率被训练
        if random.random() < 0.01:
            self.signal_type = x.signal_type
        if x.signal_type == self.signal_type:
            return d_signals[self.signal_type] + '：' + x.data
        else:
            return d_signals[self.signal_type] + '：' + '什么都没' + \
                   d_signals[self.signal_type] + '到'

In [7]:
brain = Brain()
for i in range(100):
    brain.auditory_model.process(x_see)

输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖萌!
>> 我在用耳朵（听觉皮层）听：什么都没听到
输入视觉信号：一只猫在卖

## 5.2 DNN神经网络模型

### 逻辑分类：一层神经网络

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

batch_size = 128 # 梯度下降批数据量
nb_classes = 10 # 类别
epochs = 10 # 循环训练集次数
img_size = 28 * 28 # 输入图片大小

# 加载数据，已执行shuffle-split（训练-测试集随机分割）
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 以Tensorflow为后端，归一化输入数据，生成图片向量
X_train = X_train.reshape(y_train.shape[0], img_size).astype('float32') / 255
X_test = X_test.reshape(y_test.shape[0], img_size).astype('float32') / 255

# One-Hot编码标签，将如[3,2,...] 编码成[[0,0,0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0,0,0],...]
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Using TensorFlow backend.


### 增加Hidden Layer（隐层）

In [2]:
# 创建模型，增加一层包含128个神经元节点的隐层
model = Sequential([
Dense(128, input_shape=(img_size,), activation='relu'),
Dense(10, input_shape=(128,), activation='softmax'),
])

# 编译模型
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练
model.fit(X_train, Y_train,
          batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(X_test, Y_test))

# 测试
score = model.evaluate(X_test, Y_test, verbose=0)
print('accuracy: {}'.format(score[1]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s - loss: 0.3267 - acc: 0.9108 - val_loss: 0.1945 - val_acc: 0.9432

### RELU激活函数

In [3]:
def relu(x):
    return x * (x > 0)

## 5.4 典型的DNN深层网络模型：MLP

### MLP模型

#### Keras实现MLP


In [4]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

nb_classes = 10 # 类别
img_size = 28 * 28 # 输入图片大小

# 加载数据，已执行shuffle-split（训练-测试集随机分割）
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 以Tensorflow为后端，归一化输入数据，生成图片向量
X_train = X_train.reshape(y_train.shape[0], img_size).astype('float32') / 255
X_test = X_test.reshape(y_test.shape[0], img_size).astype('float32') / 255

# One-Hot编码标签，将如[3,2,...] 编码成[[0,0,0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0,0,0],...]
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train.shape, Y_train.shape

((60000, 784), (60000, 10))

In [5]:
# 创建模型，增加一层包含128个神经元节点的隐层；Dense默认activation为linear线性
model = Sequential([
Dense(512, input_shape=(img_size,)),
Activation('relu'),
Dropout(0.2),
Dense(512, input_shape=(512,)),
Activation('relu'),
Dropout(0.2),
Dense(10, input_shape=(512,), activation='softmax'),
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total para

In [6]:
# 编译模型
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
batch_size = 128 # 梯度下降批数据量
epochs = 10 # 循环训练集次数

# 训练
model.fit(X_train, Y_train,
          batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(X_test, Y_test))

# 测试
score = model.evaluate(X_test, Y_test, verbose=0)
print('accuracy: {}'.format(score[1]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.2426 - acc: 0.9252 - val_loss: 0.1140 - val_acc: 0.9656

## 5.5 Caffe实现MLP


### 搭建MLP

见第五章目录下文件。